External summary statistics for the analyses are based on Ben Neale UK Biobank GWAS.
The are described in the following manifest file:

https://docs.google.com/spreadsheets/d/1kvPoupSzsSFBNSztMzl04xMoSC3Kcx3CrjVf4yBmESU/edit#gid=178908679

The following summary stats were used:

* 50_irnt       Standing height http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=50
* 1707_2        Handedness (chirality/laterality): Left-handed  http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=1707
* 1707_3        Handedness (chirality/laterality): Use both right and left hands equally http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=1707
* 1747_1  Hair colour (natural, before greying): Blonde   http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=1747
* 1747_4  Hair colour (natural, before greying): Dark brown       http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=1747
* 1747_5  Hair colour (natural, before greying): Black    http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=1747

``wget`` commands:


```
wget https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/additive-tsvs/50_irnt.gwas.imputed_v3.both_sexes.tsv.bgz -O 50_irnt.gwas.imputed_v3.both_sexes.tsv.bgz

wget https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/additive-tsvs/1707_2.gwas.imputed_v3.both_sexes.tsv.bgz -O 1707_2.gwas.imputed_v3.both_sexes.tsv.bgz

wget https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/additive-tsvs/1707_3.gwas.imputed_v3.both_sexes.tsv.bgz -O 1707_3.gwas.imputed_v3.both_sexes.tsv.bgz

wget https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/additive-tsvs/1747_1.gwas.imputed_v3.both_sexes.tsv.bgz -O 1747_1.gwas.imputed_v3.both_sexes.tsv.bgz

wget https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/additive-tsvs/1747_4.gwas.imputed_v3.both_sexes.tsv.bgz -O 1747_4.gwas.imputed_v3.both_sexes.tsv.bgz

wget https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/additive-tsvs/1747_5.gwas.imputed_v3.both_sexes.tsv.bgz -O 1747_5.gwas.imputed_v3.both_sexes.tsv.bgz
```

The summary statistics were then processed with https://github.com/BioPsyk/cleansumstats pipeline ``v1.6.0``, with dbSNP b156 as reference, using ``ukb_gwas/*.yaml`` files, and 
https://github.com/precimed/GWAS_SUMSTAT/blob/master/JF/run_cleansumstats_bulk.job script for pre- and post- processing of cleansumstats inputs/outputs.

Afterwards the resulting sumstats were filtered with HapMap3 SNPs:


In [ ]:
# prepare .hm3 summary stats (~1.2m SNPs)
import pandas as pd

df_hm3_rsid = pd.read_csv('imputed/w_hm3.justrs', sep='\t', header=None, names=['RSID'])

for trait in '''UKB_NEALELAB_2018_HAIR_BLACK
UKB_NEALELAB_2018_HAIR_DARK_BROWN
UKB_NEALELAB_2018_HAIR_BLONDE
UKB_NEALELAB_2018_AMBIDEXT
UKB_NEALELAB_2018_LEFTHAND
UKB_NEALELAB_2018_HEIGHT
'''.split():
    df_gwas = pd.read_csv(f'ukb_gwas/{trait}.GRCh37.gz', sep='\t')
    df_gwas = pd.merge(df_gwas, df_hm3_rsid, how='inner')
    
    # due to the way I ran cleansumstats pipeline all effect sizes were flipped (A1<->A2 mismatch)
    df_gwas['B'] = -df_gwas['B'].values
    df_gwas['Z'] = -df_gwas['Z'].values
    df_gwas.to_csv(f'ukb_gwas/{trait}.GRCh37.hm3',sep='\t', index=False)


In [ ]:
!gzip ukb_gwas/*hm3